In [85]:
import numpy as np
import pandas as pd
import datetime
import json
from sklearn import linear_model

In [86]:
import os
import socket

if os.name != "nt":
    import fcntl
    import struct

    def get_interface_ip(ifname):
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        return socket.inet_ntoa(fcntl.ioctl(s.fileno(), 0x8915, struct.pack('256s',
                                ifname[:15]))[20:24])

def get_lan_ip():
    ip = socket.gethostbyname(socket.gethostname())
    if ip.startswith("127.") and os.name != "nt":
        interfaces = [
            "eth0",
            "eth1",
            "eth2",
            "wlan0",
            "wlan1",
            "wifi0",
            "ath0",
            "ath1",
            "ppp0",
            ]
        for ifname in interfaces:
            try:
                ip = get_interface_ip(ifname)
                break
            except IOError:
                pass
    return ip

,DAY_NUM,DAY,SLOT1,SLOT2,SLOT3,SLOT4,SLOT5,SLOT6
0,0,0,69,92.0,293.0,368.0,353.0,1.0
1,1,1,90,120.0,96.0,69.0,29.0,109.0
2,2,2,87,32.0,116.0,1.0,89.0,121.0
3,3,3,96,69.0,1.0,89.0,121.0,96.0


In [74]:
    a=np.random.randint(low=1000, high=1500, size=100)
    #slot 5
    b=np.random.randint(low=1000, high=1200, size=100)
    #slot 6
    c=np.random.randint(low=800,high=1000,size=100)
    #slot 3
    d=np.random.randint(low=600,high=900,size=100)
    #slot 1
    e=np.random.randint(low=400,high=700,size=100)
    #slot 2
    f=np.random.randint(low=200,high=500,size=100)

    arr=np.array([e,f,d,a,b,c])
    data=pd.DataFrame(columns=["DAY_NUM","DAY","SLOT1","SLOT2","SLOT3","SLOT4","SLOT5","SLOT6"])
    data["DAY_NUM"]=np.arange(100)
    days=np.arange(7)
    for i in range(14):
        days=np.append(arr=days,values=np.arange(7))
    days=days[:100]    
    data["DAY"]=days
    data["SLOT1"]=e
    data["SLOT2"]=f
    data["SLOT3"]=d
    data["SLOT4"]=a
    data["SLOT5"]=b
    data["SLOT6"]=c
    x=np.array([]);
    for i in range(6):
        x=np.append(arr=a,values=data.iloc[0, 2:][i])
    
    now = datetime.datetime.now()
    hour=now.hour
    if (hour>=23 and hour<24) or (hour>0 and hour<3):
        slot=1
        from_time=3
    if hour>=3 and hour<7:
        slot=2
        from_time=7
    if hour>=7 and hour<11:
        slot=3
        from_time=11
    if hour>=11 and hour<15:
        slot=4
        from_time=15
    if hour>=15 and hour<19:
        slot=5
        from_time=19
    if hour>=19 and hour<23:
        slot=0
        from_time=23
        
    
    baseurl=get_lan_ip()+"/img"
    graphing=[0,0,0,0,0,0]
    a=[0,0,0,0,0,0]
    x=0
    nextday=0
    r=http.request('GET','http://localhost/max_bidding.php')
    js=json.loads(r.data)
#     for j in js:
#         print(j['time'])
    for i in range(6):
        train_x=np.array(data.iloc[:70,1])
        train_y=np.array(data.iloc[:70,2+slot])
        test=data.iloc[70:,1]
        import datetime
        model=linear_model.LinearRegression()
        train_x=train_x.reshape(-1,1)
        test=test.reshape(-1,1)
        model.fit(train_x,train_y)
        day=now.strftime("%w")
        y=model.predict(np.array(int(day)+nextday).reshape(-1,1))
        graphing[x]=y[0]
        for j in js:
            if j['time']==str(from_time):
                a[x]={"fromTime":from_time,"toTime":(from_time+4)%24,"urlImage":baseurl+str(slot)+".png","estTraffic":y[0],"maxBid":j['max_bid'],"minBid":j['min_bid']}
        from_time=from_time+4
        if(from_time>24):
            from_time=from_time%24
        if slot == 5:
            slot=-1
            nextday=1
        slot=slot+1
        x=x+1

    for i in range(6):
        colors=['g']*(i)+['b']+['g']*(6-i-1)
        dfs=pd.DataFrame(graphing)
        ax=dfs.plot(kind='bar',color=colors)
        img=ax.get_figure()
        img.savefig("img"+str(i)+".png")

/Users/ronaksakhuja/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:69: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/Users/ronaksakhuja/anaconda/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


70    0
71    1
72    2
73    3
74    4
75    5
76    6
77    0
78    1
79    2
80    3
81    4
82    5
83    6
84    0
85    1
86    2
87    3
88    4
89    5
90    6
91    0
92    1
93    2
94    3
95    4
96    5
97    6
98    0
99    1
Name: DAY, dtype: int64

In [20]:
json.dumps(a)

'[{"fromTime": 23, "toTime": 3, "urlImage": "192.168.43.189/img0.png", "estTraffic": 544.3892857142857}, {"fromTime": 3, "toTime": 7, "urlImage": "192.168.43.189/img1.png", "estTraffic": 359.0035714285714}, {"fromTime": 7, "toTime": 11, "urlImage": "192.168.43.189/img2.png", "estTraffic": 727.6142857142856}, {"fromTime": 11, "toTime": 15, "urlImage": "192.168.43.189/img3.png", "estTraffic": 1255.4857142857145}, {"fromTime": 15, "toTime": 19, "urlImage": "192.168.43.189/img4.png", "estTraffic": 1104.3249999999998}, {"fromTime": 19, "toTime": 23, "urlImage": "192.168.43.189/img5.png", "estTraffic": 906.4071428571428}]'

In [132]:
data=pd.read_csv("file.csv")
data.dropna(inplace=True)
x_axis=round(70/100*data.shape[0])
y_axis=data.shape[0]-x_axis


In [133]:
    now = datetime.datetime.now()
    hour=now.hour
    if (hour>=23 and hour<24) or (hour>0 and hour<3):
        slot=1
        from_time=3
    if hour>=3 and hour<7:
        slot=2
        from_time=7
    if hour>=7 and hour<11:
        slot=3
        from_time=11
    if hour>=11 and hour<15:
        slot=4
        from_time=15
    if hour>=15 and hour<19:
        slot=5
        from_time=19
    if hour>=19 and hour<23:
        slot=0
        from_time=23
        
    
    baseurl=get_lan_ip()+"/img"
    graphing=[0,0,0,0,0,0]
    a=[0,0,0,0,0,0]
    x=0
    nextday=0
    r=http.request('GET','http://localhost/max_bidding.php')
    js=json.loads(r.data)
#     for j in js:
#         print(j['time'])


In [131]:
x

0

In [134]:
    for i in range(6):
        train_x=np.array(data.iloc[:x_axis,1])
        print(train_x)

        train_y=np.array(data.iloc[:x_axis,2+slot])
        test=data.iloc[x_axis:,1]
        import datetime
        model=linear_model.LinearRegression()
        train_x=train_x.reshape(-1,1)
        test=test.reshape(-1,1)
        model.fit(train_x,train_y)
        day=now.strftime("%w")
        y=model.predict(np.array(int(day)+nextday).reshape(-1,1))
        graphing[x]=y[0]
        for j in js:
            if j['time']==str(from_time):
                a[x]={"fromTime":from_time,"toTime":(from_time+4)%24,"urlImage":baseurl+str(slot)+".png","estTraffic":y[0],"maxBid":j['max_bid'],"minBid":j['min_bid']}
        from_time=from_time+4
        if(from_time>24):
            from_time=from_time%24
        if slot == 5:
            slot=-1
            nextday=1
        slot=slot+1
        x=x+1


    for i in range(6):
        colors=['g']*(i)+['b']+['g']*(6-i-1)
        dfs=pd.DataFrame(graphing)
        ax=dfs.plot(kind='bar',color=colors)
        img=ax.get_figure()
        img.savefig("img"+str(i)+".png")

[0 1 2 3 4 5 6 0 1 2 3 4 5]
[0 1 2 3 4 5 6 0 1 2 3 4 5]
[0 1 2 3 4 5 6 0 1 2 3 4 5]
[0 1 2 3 4 5 6 0 1 2 3 4 5]
[0 1 2 3 4 5 6 0 1 2 3 4 5]
[0 1 2 3 4 5 6 0 1 2 3 4 5]


/Users/ronaksakhuja/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.
/Users/ronaksakhuja/anaconda/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [135]:
a

[{'estTraffic': 187.97342192691028,
  'fromTime': 7,
  'maxBid': '3001',
  'minBid': '2950',
  'toTime': 11,
  'urlImage': '192.168.43.189/img2.png'},
 {'estTraffic': 119.75415282392026,
  'fromTime': 11,
  'maxBid': '4400',
  'minBid': '3780',
  'toTime': 15,
  'urlImage': '192.168.43.189/img3.png'},
 {'estTraffic': 167.62790697674419,
  'fromTime': 15,
  'maxBid': '3500',
  'minBid': '3500',
  'toTime': 19,
  'urlImage': '192.168.43.189/img4.png'},
 {'estTraffic': 127.40531561461793,
  'fromTime': 19,
  'maxBid': '3010',
  'minBid': '3010',
  'toTime': 23,
  'urlImage': '192.168.43.189/img5.png'},
 {'estTraffic': 148.29568106312294,
  'fromTime': 23,
  'maxBid': '1000',
  'minBid': '2500',
  'toTime': 3,
  'urlImage': '192.168.43.189/img0.png'},
 {'estTraffic': 142.95847176079732,
  'fromTime': 3,
  'maxBid': '27510',
  'minBid': '2750',
  'toTime': 7,
  'urlImage': '192.168.43.189/img1.png'}]